In [1]:
#import the necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import missingno as msno
from IPython.display import display
from sklearn.preprocessing import OrdinalEncoder
sns.set()
import warnings
warnings.filterwarnings('ignore')

In [14]:
#load the dataset
data = pd.read_csv('./DentalData311.csv')
data.sample(5)

,Timestamp,Patient Number,Chief complaint,Nature of Pain,Severity of pain,Onset and mode of pain,Factors which worsens the pain,Is the swelling painful?,Has the swelling changed since it was first noticed? If yes how quickly?,"Does the swelling changes during normal activities such as eating, speaking, etc?",...,Is there bleeding from the ulcer,Is there discharge from the ulcer?,Is there a foul smell from the ulcer?,Do the ulcers interfere with daily activities,Has the ulcer changed since first noticed?,Have you had similar ulcers?,Final Diagnosis,Is there bleeding in the gums?,Is there pain in the gums,"If any tooth/teeth is/are mobile, what is the degree of mobility"
5,31/10/2023 09:39:07,6,Ulecer,throbbing,Moderate,The pain came suddenly,Hot foods or drinks,No,No,No,...,No,No,No,No,No,No,Acute pulpits,No,No,No
12,03/11/2023 09:08:45,F70,No,No,No,No,No,No,No,No,...,No,No,No,No,No,No,Chronic destructive periodontitis,No,Yes,2
35,03/11/2023 11:04:05,F47,Ulecer,burning,Severe,The pain came suddenly,No,No,No,No,...,No,No,No,Yes,No,No,Aphthous ulcer,No,No,No
39,03/11/2023 11:10:34,F32,Pain,aching,Mild,lingering pain,Laying down,No,No,No,...,No,No,No,No,No,No,Acute pulpits,No,No,No
37,03/11/2023 11:06:09,M9,Swelling,throbbing,Moderate,lingering pain,Eating,No,Slow,No,...,No,No,No,No,No,No,Pulpits with swelling,No,No,No


In [15]:
#Dropping unnecessary fields
data.drop('Timestamp',axis=1, inplace=True)
data.drop('Patient Number',axis=1, inplace=True)
data.sample(5)

,Chief complaint,Nature of Pain,Severity of pain,Onset and mode of pain,Factors which worsens the pain,Is the swelling painful?,Has the swelling changed since it was first noticed? If yes how quickly?,"Does the swelling changes during normal activities such as eating, speaking, etc?",Is the ulcer painful,Is there bleeding from the ulcer,Is there discharge from the ulcer?,Is there a foul smell from the ulcer?,Do the ulcers interfere with daily activities,Has the ulcer changed since first noticed?,Have you had similar ulcers?,Final Diagnosis,Is there bleeding in the gums?,Is there pain in the gums,"If any tooth/teeth is/are mobile, what is the degree of mobility"
43,Swelling,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Chronic pulpits with swelling,No,No,No
36,Pain,throbbing,Severe,lingering pain,Hot foods or drinks,No,No,No,No,No,No,No,No,No,No,Pulpits,No,No,No
27,Ulecer,stabbing,Moderate,No,No,No,No,No,Yes,No,No,No,Yes,Yes,Yes,Aphthous ulcer,No,No,No
19,Swelling,aching,Mild,NaN,Laying down,No,Rapid,No,No,No,No,No,No,No,No,Acute pulpits with extra oral swelling,No,No,No
14,Pain,stabbing,Severe,No,Laying down,No,No,No,No,No,No,No,No,No,No,Acute pulpits,No,No,No


In [17]:
#using ordinal encoder to change the fields to numerical values
encode=OrdinalEncoder(dtype=float)
patient_encode=pd.DataFrame(encode.fit_transform(data),columns=data.columns)
patient_encode.head(5)

,Chief complaint,Nature of Pain,Severity of pain,Onset and mode of pain,Factors which worsens the pain,Is the swelling painful?,Has the swelling changed since it was first noticed? If yes how quickly?,"Does the swelling changes during normal activities such as eating, speaking, etc?",Is the ulcer painful,Is there bleeding from the ulcer,Is there discharge from the ulcer?,Is there a foul smell from the ulcer?,Do the ulcers interfere with daily activities,Has the ulcer changed since first noticed?,Have you had similar ulcers?,Final Diagnosis,Is there bleeding in the gums?,Is there pain in the gums,"If any tooth/teeth is/are mobile, what is the degree of mobility"
0,1.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,1.0
1,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,0.0,1.0
2,2.0,4.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1.0
3,1.0,4.0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
4,2.0,0.0,2.0,0.0,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,0.0,1.0
